<a href="https://colab.research.google.com/github/iamharshvishwakarma/Rocket-Failure-Prediction-model/blob/main/rocket_failure_prediction_using_thrust_weight_ratio_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Loading and Preprocessing the Data

In [7]:
# Load the dataset
df = pd.read_csv("SpaceMissions.csv")

# Select relevant features (Liftoff Thrust, Payload Mass, and Mission Status)
features = ["Liftoff Thrust (kN)", "Payload Mass (kg)", "Mission Status"]
df = df[features]

# Convert columns to numeric values, coercing errors to NaN
df["Liftoff Thrust (kN)"] = pd.to_numeric(df["Liftoff Thrust (kN)"], errors="coerce")
df["Payload Mass (kg)"] = pd.to_numeric(df["Payload Mass (kg)"], errors="coerce")

# Drop rows with missing values in the selected features
df = df.dropna()

# Handle zero or near-zero Payload Mass
df = df[df["Payload Mass (kg)"] > 0]  # Filter out rows where Payload Mass is zero

# Calculate Thrust-to-Weight Ratio
df["Thrust-to-Weight Ratio"] = df["Liftoff Thrust (kN)"] / (df["Payload Mass (kg)"] * 9.81)

# Handle infinite or extremely large values
df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace infinite values with NaN
df = df.dropna()  # Drop rows with NaN values (infinite or invalid ratios)

# Clip extremely large values to a reasonable range (optional)
df["Thrust-to-Weight Ratio"] = df["Thrust-to-Weight Ratio"].clip(lower=-1e10, upper=1e10)

# Encode the target variable (Mission Status: Success/Failure)
df["Mission Status"] = df["Mission Status"].apply(lambda x: 1 if x == "Failure" else 0)

# Split the data into features (X) and target (y)
X = df[["Thrust-to-Weight Ratio"]]
y = df["Mission Status"]

Spliting the Data into Training and Testing Sets

In [8]:
# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Training the Decision Tree Model

In [9]:
# Initialize the Decision Tree Classifier
model = DecisionTreeClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Takeing user input and makeing predictions

In [11]:
# Function to take user input and make predictions
def predict_rocket_status():
    # Take input from the user
    liftoff_thrust = float(input("Enter Liftoff Thrust (kN): "))
    payload_mass = float(input("Enter Payload Mass (kg): "))

    # Handle zero or near-zero Payload Mass
    if payload_mass <= 0:
        print("Error: Payload Mass must be greater than 0.")
        return

    # Calculate Thrust-to-Weight Ratio
    thrust_to_weight_ratio = liftoff_thrust / (payload_mass * 9.81)

    # Handle extremely large values (optional)
    thrust_to_weight_ratio = np.clip(thrust_to_weight_ratio, -1e10, 1e10)

    # Create a DataFrame for the input data
    user_input = pd.DataFrame({
        "Thrust-to-Weight Ratio": [thrust_to_weight_ratio]
    })

    # Make a prediction using the trained model
    prediction = model.predict(user_input)

    # Display the result
    if prediction[0] == 1:
        print("Prediction: Rocket Launch Failure")
    else:
        print("Prediction: Rocket Launch Success")

# Call the function to take input and make predictions
predict_rocket_status()

Enter Liftoff Thrust (kN): 5000
Enter Payload Mass (kg): 10000
Prediction: Rocket Launch Success
